# Required Imports & Functions

In [ ]:
import astroquery
import datetime
import pytz
import matplotlib.pyplot as plt 
import pandas as pd 
import astropy 
from astropy.table import Table
from astropy.coordinates import SkyCoord, name_resolve
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astroquery.simbad import Simbad
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Function to retrieve the distance to a celestial object using Simbad catalog data
def get_obj_distance(obj_name):
    """
    Query the Simbad astronomical database for the parallax of a given object
    and calculate its distance.

    Parameters:
        obj_name (str): Name of the celestial object to query (e.g., "Sirius").

    Returns:
        float: Distance to the object in light-years. Returns 0 if parallax is unavailable.
    """
    # Add parallax field to Simbad queries
    Simbad.add_votable_fields('parallax')

    # Query Simbad for the object
    result = Simbad.query_object(obj_name)
    parallax_mas = result['plx_value'][0]  # Parallax in milliarcseconds (mas)

    # Convert parallax to distance: distance (pc) = 1000 / parallax (mas)
    if parallax_mas:
        distance_pc = 1000.0 / parallax_mas  # Distance in parsecs
        return distance_pc * 3.26156  # Convert parsecs to light-years
    else:
        print("Parallax not available for object:", obj_name)
        return 0


# Function to add stars to an existing Messier astropy table
def add_star_data(messier_table, star_names):
    """
    Add additional star data to an existing Messier object table.

    Parameters:
        messier_table (Astropy Table): Original table of Messier objects.
        star_names (list of str): List of star names to add to the table.

    Returns:
        Astropy Table: A new table including the original Messier objects
                       plus the newly added stars. Each added star includes:
                       - Name
                       - Object type
                       - Distance in kiloparsecs
                       - RA/DEC in degrees
                       - Placeholder values for missing fields
    """
    new_table = messier_table.copy()  # Avoid modifying original table

    # Loop through the provided star names
    for i, obj_name in enumerate(star_names):
        # Get RA/DEC coordinates using SkyCoord
        c = SkyCoord.from_name(obj_name)
        RA = float(c.ra.degree)    # Right Ascension in degrees
        DEC = float(c.dec.degree)  # Declination in degrees

        # Get object distance in light-years, then convert to kiloparsecs
        obj_dist = get_obj_distance(obj_name) / 1000  # kly units (kilolight-years)

        # Add a new row to the table with star info
        new_table.add_row([
            "star_%s" % i,        # Internal ID
            "star_%s" % i,        # Another placeholder ID
            obj_name,             # Object name
            "Binary or Multi-Star", # Classification
            "Star",               # Object type
            obj_dist,             # Distance (kly)
            0,                    # Placeholder for magnitude or other property
            RA,                   # Right Ascension (deg)
            DEC,                  # Declination (deg)
            "NA"                  # Placeholder for additional info
        ])

    return new_table


# Convert Right Ascension from hours, minutes, seconds to decimal degrees
def RA_convert(hours, minutes, seconds):
    """
    Convert Right Ascension (RA) from HMS format to decimal degrees.

    Parameters:
        hours (int or float): Hours component of RA.
        minutes (int or float): Minutes component of RA.
        seconds (float): Seconds component of RA.

    Returns:
        float: RA in decimal degrees (0-360°).
    """
    degree_unit = (((((seconds / 60) + minutes) / 60) + hours) / 24) * 360
    return degree_unit


# Convert Declination from degrees, arcminutes, arcseconds to decimal degrees
def DEC_convert(deg, arcmin, arcsec):
    """
    Convert Declination (DEC) from DMS format to decimal degrees.

    Parameters:
        deg (int or float): Degrees component of DEC.
        arcmin (int or float): Arcminutes component of DEC.
        arcsec (float): Arcseconds component of DEC.

    Returns:
        float: DEC in decimal degrees (-90 to +90°).
    """
    degree_unit = ((((arcsec / 60) + arcmin) / 60) + deg)
    return degree_unit

def is_object_above_horizon(ra, dec, lat, lon, timezone, name, local_time=None):
    """
    Determine whether a celestial object is above the horizon for a given observer location and time.

    Parameters:
        ra (float): Right Ascension of the object in degrees.
        dec (float): Declination of the object in degrees.
        lat (float): Observer's latitude in degrees.
        lon (float): Observer's longitude in degrees.
        timezone (str): Observer's local timezone (e.g., "America/Halifax").
        name (str): Name of the celestial object (for display/logging purposes).
        local_time (datetime, optional): The local datetime at which to check visibility.
                                         Defaults to current time if None.

    Returns:
        tuple: 
            - int: 1 if the object is comfortably above horizon (altitude >= 15°),
                   0 if below comfortable viewing altitude.
            - float: altitude of the object in degrees (if above threshold), otherwise 0.
    """
    # Get the local timezone object
    local_tz = pytz.timezone(timezone)
    
    # If no local time is provided, use current time
    if local_time is None:
        local_time = datetime.datetime.now(local_tz)
    else:
        # Ensure provided datetime is timezone-aware
        if local_time.tzinfo is None:
            local_time = local_tz.localize(local_time)
        else:
            local_time = local_time.astimezone(local_tz)

    # Convert local time to UTC for astropy
    utc_time = local_time.astimezone(pytz.utc)
    time = Time(utc_time)

    # Define the observer's location on Earth
    location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)

    # Convert the object's RA/Dec coordinates to Alt/Az for this observer/time
    sky_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')
    altaz = sky_coord.transform_to(AltAz(obstime=time, location=location))

    # Extract the altitude (angle above the horizon)
    altitude = altaz.alt.deg

    # Thresholds for visibility classification
    DIV = 15  # degrees above horizon considered comfortably visible
    if altitude >= DIV:
        #print("%s - YES (%.2f deg)" % (name, altitude))
        return 1, altitude  # Object is well above the horizon
    elif 1 < altitude < DIV:
        #print("%s - maybe (%.2f deg)" % (name, altitude))
        return 0, 0  # Object is marginally visible
    else:
        #print("%s - NO (%.2f deg)" % (name, altitude))
        return 0, 0  # Object is below horizon


def check_objects(objects, local_time):
    """
    Filter a table of celestial objects to determine which are currently viewable.

    Uses a fixed observer location (Halifax) and local time to compute visibility.

    Parameters:
        objects (Astropy Table or similar structure): Table containing object info,
            must include 'RA', 'DEC', and 'MESSIER_NUM' columns.
        local_time (datetime): Local datetime at which to check visibility.

    Returns:
        Astropy Table: Filtered table containing only objects that are above
                       the comfortable altitude threshold. Adds an 'altitude' column.
    """
    # Observer location for Halifax
    lat_halifax = 44.636
    long_halifax = -63.5917
    timezone = "America/Halifax"

    altitude_list = []     # Store computed altitudes of visible objects
    good_name_list = []    # Store names of objects that are comfortably visible

    # Iterate over all objects in the table
    for i in range(len(objects)):
        ra = objects["RA"][i]
        dec = objects["DEC"][i]
        name = objects["MESSIER_NUM"][i]

        # Check if object is above horizon
        checker, altitude = is_object_above_horizon(ra, dec, lat_halifax, long_halifax, timezone, name, local_time)

        # If comfortably visible, store name and altitude
        if checker == 1:
            good_name_list.append(name)
            altitude_list.append(altitude)

    # Create a set of visible object names for filtering
    ids_set = set(good_name_list)

    # Filter the original table to include only visible objects
    filtered_table = objects[[row['MESSIER_NUM'] in ids_set for row in objects]]

    # Add the altitude column
    filtered_table["altitude"] = altitude_list

    return filtered_table


def single_object_type(TABLE, obj_type):
    # Function splits final observation targets list into categories (e.g., galaxy, nebulae, etc.)
    new_table = TABLE[TABLE["OBJ_CATEGORY"]==obj_type]
    return new_table


def save_tables_to_pdf(tables, pdf_filename):
    """
    Save one or more Astropy tables to a multi-page PDF, with each table on its own page.

    Parameters:
        tables (list of tuples): Each tuple should be (table_name, astropy_table), where:
            - table_name (str): Title to display above the table.
            - astropy_table (Astropy Table): Table object to save.
        pdf_filename (str): Path and filename of the output PDF file.

    Behavior:
        - Each table is rendered as monospaced text in a standard 8.5" x 11" page.
        - Large tables are fully displayed (no truncation) using table.pformat(max_lines=-1, max_width=-1).
        - Saves all pages to a single PDF using matplotlib.backends.backend_pdf.PdfPages.
    """
    # Open a PDF file to save multiple pages
    with PdfPages(pdf_filename) as pdf:
        # Loop through all tables provided
        for name, table in tables:
            # Create a new figure for each table (standard US Letter size)
            fig, ax = plt.subplots(figsize=(8.5, 11))
            ax.axis('off')  # Hide axes for clean text display

            # Convert the Astropy table to formatted string
            # max_lines=-1 and max_width=-1 ensures full table is printed
            table_str = table.pformat(max_lines=-1, max_width=-1)
            text = "\n".join(table_str)

            # Add table text to the figure, with table title
            ax.text(
                0, 1,                    # Top-left corner of the page
                f"{name}\n\n{text}",     # Table title + content
                fontsize=10,             # Readable font size
                va='top',                # Align text from the top
                family='monospace'       # Monospaced font preserves table formatting
            )

            # Save the current figure as a page in the PDF
            pdf.savefig(fig, bbox_inches='tight')

            # Close the figure to free memory
            plt.close(fig)


# Import Messier Object List

In [ ]:
# Load messier list file + convert to astropy table
messier_list_file = "Messier Objects.xlsx"
messier_objects = Table.from_pandas(pd.read_excel(messier_list_file))
messier_objects.sort('MESSIER_NUM')
print("Messier Catalog Loaded")

# Add Stars or other objects

In [ ]:
### Example star names that may be good for viewing ###
'''
star_names = ["Albireo", "Sirius", "Alpha Centauri","Capella","Antares","Castor","Fomalhaut","Arcturus","Betelgeuse",
              "Dubhe","Nunki","Rasalhague","Mizar","Sabik","Sarin","Alcor", "Alphecca","Izar","Rasalgethi", "Sheliak"]
'''
########################################################################
star_names = ["Albireo", "Sheliak", "Mizar", "Rasalgethi"]
star_names.sort()
observation_targets = add_star_data(messier_objects, star_names)
print("Stars Added")

# Objects in View Tonight

In [ ]:
### Set your desired observation time ###
desired_time = datetime.datetime(2026, 1, 28, 19, 45, 0) # Year, Month, Day, Hour, Min, Sec

### Run main functions to determine objects in view tonight ###
objects_in_view_tonight = check_objects(observation_targets, desired_time)
print("Objects Identified")

# Specific Type of Object

In [ ]:
###########################################################################
observable_GC = single_object_type(objects_in_view_tonight, "Globular cluster")
observable_GC.sort("APP_MAG")
###########################################################################
observable_OC = single_object_type(objects_in_view_tonight, "Open cluster")
observable_OC.sort("APP_MAG")
###########################################################################
observable_nebula = single_object_type(objects_in_view_tonight, "Nebula")
observable_nebula.sort("APP_MAG")
###########################################################################
observable_galaxy = single_object_type(objects_in_view_tonight, "Galaxy")
observable_galaxy.sort("APP_MAG")
###########################################################################
observable_star = single_object_type(objects_in_view_tonight, "Star")

In [ ]:
observable_GC

In [ ]:
observable_OC

In [ ]:
observable_nebula

In [ ]:
observable_star

In [ ]:
observable_galaxy

# Save to PDF

In [ ]:
save_name = "C:\\Users\\xboxa\\Downloads\\BGO_observation_targets_Jan28"  # change to your desired directory path
########################################################################
save_tables = [("observable_star", observable_star), ("observable_OC", observable_OC), 
          ("observable_nebula", observable_nebula), ("observable_GC", observable_GC), ("observable_galaxy", observable_galaxy)]
save_tables_to_pdf(save_tables, "%s.pdf"%save_name)